<a href="https://colab.research.google.com/github/nicologhielmetti/AN2DL-challenges/blob/master/challenge1/vgg_19_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [2]:
!pip install gdown
!gdown https://drive.google.com/uc?id=1Mv7vKoI-QL6kV-1TIDE7N67_L0LXvJAg
!unzip /content/ANDL2.zip

Downloading...
From: https://drive.google.com/uc?id=1Mv7vKoI-QL6kV-1TIDE7N67_L0LXvJAg
To: /home/nicolo/PycharmProjects/AN2DL-challenges/challenge1/ANDL2.zip
433MB [09:40, 745kB/s]  
unzip:  cannot find or open /content/ANDL2.zip, /content/ANDL2.zip.zip or /content/ANDL2.zip.ZIP.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
import json
import os
import shutil
from datetime import datetime
from functools import partial

from PIL import Image

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorboard import program

SEED = 1996

In [ ]:
def divideDatasetInTargetFolders(json_definition, dataset_path):
    for elem in json_definition:
        dest_dir = os.path.join(dataset_path, str(json_definition[elem]))
        if not os.path.isdir(dest_dir):
            os.mkdir(dest_dir)
        try:
            shutil.move(os.path.join(dataset_path, elem),
                        os.path.join(dest_dir, elem)
                        )
        except FileNotFoundError as e:
            print("File not found: " + str(e))
            continue
    os.mkdir(os.path.join(dataset_path, "augmented"))
    os.mkdir(os.path.join(dataset_path, "augmented/training"))
    os.mkdir(os.path.join(dataset_path, "augmented/validation"))


def getMaxImageSize(dataset_dir):
    max_w = 0
    max_h = 0
    path = os.path.join(os.getcwd(), dataset_dir)
    for filename in os.listdir(path):
        if filename.endswith(".jpg"):
            image = Image.open(os.path.join(path, filename))
            width, height = image.size
            max_w = width if width > max_w else max_w
            max_h = height if height > max_h else max_h
        else:
            print("This file -> " + filename + " is not .jpg")
    return max_w, max_h


def getMinImageSize(dataset_dir, max_w, max_h):
    min_w = max_w
    min_h = max_h
    for filename in os.listdir(dataset_dir):
        if filename.endswith(".jpg"):
            image = Image.open(os.path.join(dataset_dir, filename))
            width, height = image.size
            min_w = width if width < min_w else min_w
            min_h = height if height < min_h else min_h
        else:
            print("This file -> " + filename + " is not .jpg")
    return min_w, min_h

In [ ]:
train_path = os.path.join(os.getcwd(), 'MaskDataset/training')
test_path  = os.path.join(os.getcwd(), 'MaskDataset/test')

In [ ]:
division_dict = json.load(
  open(os.path.join(os.getcwd(), 'MaskDataset/train_gt.json'))
)

divideDatasetInTargetFolders(division_dict, train_path)

In [ ]:
# remember to check both train and test datasets to be sure of max dimensions
max_w, max_h = max(getMaxImageSize(os.path.join(train_path, '0')),
                   getMaxImageSize(os.path.join(train_path, '1')),
                   getMaxImageSize(os.path.join(train_path, '2')))
print("Maximum width and height: " + str((max_w, max_h)))

min_w, min_h = min(getMinImageSize(os.path.join(train_path, '0'), max_w, max_h),
                   getMinImageSize(os.path.join(train_path, '1'), max_w, max_h),
                   getMinImageSize(os.path.join(train_path, '2'), max_w, max_h))
print("Minimum width and height:  " + str((min_w, min_h)))
print("Maximum width  expansion:  " + str(max_w - min_w) + ", increase ratio: " +
      str(float(max_w) / float(max_w - min_w)))
print("Maximum height expansion:  " + str(max_h - min_h) + ", increase ratio: " +
      str(float(max_h) / float(max_h - min_h)))

In [ ]:
preproc_fun_fixed = partial(tf.keras.preprocessing.image.smart_resize, size=(max_w, max_h))

train_data_gen = ImageDataGenerator(rotation_range=10,
                                    width_shift_range=10,
                                    height_shift_range=10,
                                    zoom_range=0.3,
                                    horizontal_flip=True,
                                    fill_mode='reflect',
                                    rescale=1. / 255,
                                    validation_split=0.3,
                                    preprocessing_function=preproc_fun_fixed
                                    )

test_data_gen = ImageDataGenerator(rescale=1. / 255, preprocessing_function=preproc_fun_fixed)

classes = ['0', '1', '2']
save_dir = os.path.join(train_path, 'augmented')

import pandas as pd
images = [f for f in os.listdir(test_path)]
images = pd.DataFrame(images)
images.rename(columns = {0:'filename'}, inplace = True)
images["class"] = 'test'

bs = 32

train_gen = train_data_gen.flow_from_directory(train_path,
                                               target_size=(max_w, max_h),
                                               seed=SEED,
                                               classes=classes,
                                               #save_prefix='training_aug',
                                               #save_to_dir=os.path.join(save_dir, 'training'),
                                               subset='training',
                                               shuffle=True,
                                               batch_size=bs
                                               )

valid_gen = train_data_gen.flow_from_directory(train_path,
                                               target_size=(max_w, max_h),
                                               seed=SEED,
                                               classes=classes,
                                               #save_prefix='validation',
                                               #save_to_dir=os.path.join(save_dir, 'validation'),
                                               subset='validation',
                                               shuffle=False,
                                               batch_size=bs
                                               )

test_gen = test_data_gen.flow_from_dataframe(images,
                                               test_path,
                                               batch_size=bs,
                                               target_size=(max_h, max_w),
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)

# set the right order for predictions
test_gen.reset()

train_set = tf.data.Dataset.from_generator(lambda: train_gen,
                                           output_types=(tf.float32, tf.float32),
                                           output_shapes=(
                                               [None, max_w, max_h, 3],
                                               [None, len(classes)]
                                           ))

validation_set = tf.data.Dataset.from_generator(lambda: valid_gen,
                                                output_types=(tf.float32, tf.float32),
                                                output_shapes=(
                                                    [None, max_w, max_h, 3],
                                                    [None, len(classes)]
                                                ))

test_set = tf.data.Dataset.from_generator(lambda: test_gen,
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes=(
                                              [None, max_w, max_h, 3],
                                              [None, len(classes)]
                                          ))

train_set.repeat()
validation_set.repeat()
test_set.repeat()

In [ ]:
vgg = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=(max_h, max_w, 3))
vgg.summary()